Загружаем и предобрабатываем данные

In [1]:
import pandas as pd

data = pd.read_csv('PsyHack_RUDN_train.csv', sep='\t')
data.head()

,ID,label,text
0,307,Non-depression,С детства я люблю футбол. Самые теплые воспоми...
1,243,Non-depression,Каждому человеку нужен мир. Мир- это весь земн...
2,73,Non-depression,"Я, другие и мир. Да, именно в таком порядке ме..."
3,301,Non-depression,Позиционирование себя как полноценного человек...
4,60,Non-depression,"Я, другие и мир. Такая неопределенная тема. Мы..."


Очищаем текст

Чтобы анализ текста был наиболее точен нужно удалить слова, не влияющие на смысл текста: предолги, союзы, собственные имена, другие служебные части речи и знаки пунктуации

In [2]:
import pymorphy2, re

In [3]:
morph = pymorphy2.MorphAnalyzer()

In [4]:
def clean_text(text):
    new_text = ''
    grams_exclusion = {'PREP', 'CONJ', 'INTJ', 'Name', 'Surn', 'Patr', 'Orgn', 'Trad'}
    text = re.sub('[^а-яА-Яa-zA-ZёЁ]', ' ', text)

    words = text.split()
    for word in words:
        p = morph.parse(word)[0]
        if not any(tag in p.tag for tag in grams_exclusion):
            new_text += ' ' + p.normal_form
    new_text = new_text[1:]
    return new_text

In [5]:
data['text_lem'] = data['text'].apply(lambda x: clean_text(x))

In [6]:
data.head()

,ID,label,text,text_lem
0,307,Non-depression,С детства я люблю футбол. Самые теплые воспоми...,детство я любить футбол самый тёплый воспомина...
1,243,Non-depression,Каждому человеку нужен мир. Мир- это весь земн...,каждый человек нужный мир мир это весь земной ...
2,73,Non-depression,"Я, другие и мир. Да, именно в таком порядке ме...",я другой мир да именно такой порядок я устраив...
3,301,Non-depression,Позиционирование себя как полноценного человек...,позиционирование себя полноценный человек восп...
4,60,Non-depression,"Я, другие и мир. Такая неопределенная тема. Мы...",я другой мир такой неопределённый тема мысль с...


Так как мы хотим уловить общий смысл эссе можно воспользоваться метрикой TF-IDF, которая будет сопоставлять каждому слову его коэффициент смысловой значимости. Значимость оценивается количеством встечаемости слова в документе, но так же нужно учесть размер документов, так как в больших документов большая вероятность встретить какое нибудь слово несколько раз.

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(ngram_range=(1,2), lowercase=True, min_df=2, max_df=0.2, encoding='cp1251')
vec.fit(data['text_lem'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='cp1251',
                input='content', lowercase=True, max_df=0.2, max_features=None,
                min_df=2, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [8]:
from sklearn.preprocessing import LabelEncoder

X = vec.transform(data['text_lem'])
lenc = LabelEncoder()
y = lenc.fit_transform(data['label'])

In [22]:
import numpy as np

alphas = np.logspace(-12, -7, 80)

Строим классификатор 

In [10]:
from sklearn.model_selection import validation_curve
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

sgd_logit = SGDClassifier(loss='modified_huber', random_state=1, n_jobs=-1, max_iter=2000)
logit_pipe = Pipeline([('sgd_logit', sgd_logit)])

In [11]:
val_train, val_test = validation_curve(logit_pipe, X, y, 'sgd_logit__alpha', alphas, cv=4, scoring='f1_micro')

In [12]:
best_alpha = alphas[val_test.mean(1).argmax()]

In [13]:
from matplotlib import pyplot as plt

def plot_with_err(x, data, **kwargs):
    mu, std = data.mean(1), data.std(1)
    lines = plt.plot(x, mu, '-', **kwargs)
    plt.fill_between(x, mu - std, mu + std, edgecolor='none',
    facecolor=lines[0].get_color(), alpha=0.2)

Обучаем модель

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=1)

In [16]:
model = SGDClassifier(loss='modified_huber', random_state=1, n_jobs=-1,
                          alpha=best_alpha, verbose=False).fit(X_train, y_train)

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

y_pred = model.predict(X_holdout)
print('Accuracy - {0:.1%}'.format(accuracy_score(y_holdout, y_pred)))
print(classification_report(y_holdout, y_pred, target_names=lenc.classes_))

Accuracy - 77.6%
                precision    recall  f1-score   support

    Depression       0.83      0.43      0.57        23
Non-depression       0.76      0.95      0.85        44

      accuracy                           0.78        67
     macro avg       0.80      0.69      0.71        67
  weighted avg       0.79      0.78      0.75        67



Используем bag-of-words в качестве признаков.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['text_lem']).toarray()
y = data['label'] == 'Depression'

Будем обучать два простых алгоритма: <a href='http://www.machinelearning.ru/wiki/index.php?title=%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D0%BE%D0%BF%D0%BE%D1%80%D0%BD%D1%8B%D1%85_%D0%B2%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D0%BE%D0%B2'>SVM</a> и <a href='https://towardsdatascience.com/the-random-forest-algorithm-d457d499ffcd'>Random Forest</a>.

In [19]:
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score

RANDOM_STATE = 1337

rf_parameters = {
    'n_estimators': (10, 100),
    'max_features': ('sqrt', 'log2', None),
    'max_depth': (None, 3, 10),
    'min_samples_split': (2, 5),
    'min_samples_leaf': (1, 3)
}

svm_parameters = {
    'C': (0.01, 0.1, 1, 10),
}

grid_parameters = {
    'scoring': make_scorer(roc_auc_score),
    'n_jobs': -1,
    'cv': 3,
    'iid': True,
    'return_train_score': True
}

Инициализируем модели и сетки для подбора параметров

In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


rf = RandomForestClassifier(n_jobs=-1, random_state=RANDOM_STATE)
svm = SVC(random_state=RANDOM_STATE, probability=True)

grid_rf = GridSearchCV(
    estimator=rf,
    param_grid=rf_parameters,
    scoring=make_scorer(roc_auc_score),
    n_jobs=-1,
    cv=3,
    iid=True,
    return_train_score=True
)

grid_svm = GridSearchCV(
    estimator=svm,
    param_grid=svm_parameters,
    scoring=make_scorer(roc_auc_score),
    n_jobs=-1,
    cv=3,
    iid=True,
    return_train_score=True
)

Обучаем модели и оцениваем точность

In [21]:
grid_rf.fit(X, y)
grid_svm.fit(X, y)

print('Random Forest ROC AUC score: ', grid_rf.best_score_)
print('SVM ROC AUC score: ', grid_svm.best_score_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Random Forest ROC AUC score:  0.6622608907356624
SVM ROC AUC score:  0.6890056178773843
